# Part 4: Machine Learning
In this part I will use machine learning to verify my research.
I have chosen in Supervised Learning.
My goal is to predict the scale of tornado storms.



Import necessary packages

In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import linear_model, metrics, preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import r2_score, accuracy_score, classification_report
from sklearn.metrics import make_scorer

First of all, I need to convert categorial values to numeric values.

In [2]:
# This function convert str values of country column to numeric values
def convertCountryToNumber(df):
    list_of_country = df['Country'].unique().tolist()
    a_list = list(range(0, len(list_of_country)))
    dictionry = dict(zip(list_of_country, range(len(a_list))))
    df.replace({'Country': dictionry }, inplace=True)

In [3]:
df = pd.read_csv("TornadoTableAfterDataCleaning.csv")
convertCountryToNumber(df)
df.to_csv('new.csv', index=False, header=True)
df = pd.read_csv("new.csv")
print("The df after convert categorical values to numeric values")
df

The df after convert categorical values to numeric values


,Scale (EF),Years,Length (KM),Width (KM),Country,Latitude,Longtitude,Deaths
0,1,2022,3.235,0.091,0,40.3790,-75.2266,0
1,0,2022,0.209,0.046,1,38.9196,-77.2259,0
2,0,2022,0.080,0.027,1,38.8653,-77.4579,0
3,1,2022,14.709,0.457,0,41.1600,-76.7300,1
4,1,2022,0.354,0.091,2,35.8672,-78.8372,0
...,...,...,...,...,...,...,...,...
14785,1,2010,1.271,0.137,10,31.5095,-93.9232,0
14786,0,2010,0.257,0.023,6,30.8150,-92.0455,0
14787,1,2010,7.435,0.023,29,33.7160,-118.1250,0
14788,0,2010,0.225,0.009,29,34.4248,-119.8770,0


Before the machine learning algorithm, I need to choose the Training_Features and the target_column.
In my case, the Training_Features is all dataframe except for scale (EF) column and the target_column is the scale (EF) column.

In [4]:
# This function return X that represent Training_Features and y that represent target_column
def load_dataset(df, target_column):
    Training_Features = df.columns[df.columns != target_column]
    X = df[Training_Features]
    y = df[target_column]
    return X, y

After I get the values X and y I need split the data to X_train, X_test, y_train, y_test 

In [5]:
# This function get X and y and return X_train, X_test, y_train, y_test after split
def split_to_train_and_test(X, y, test_ratio, rand_state):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio, random_state=rand_state)
    return X_train, X_test, y_train, y_test

In [6]:
# This function return the predicted value
def predictSt(trained_1st_model, X_test):
    predicted_vals = trained_1st_model.predict(X_test)
    return predicted_vals

In [7]:
target_col_name = 'Scale (EF)'
X, y = load_dataset(df, target_col_name)
print("X and y:\n")
print(X, y)
X_train, X_test, y_train, y_test = split_to_train_and_test(X, y, 0.3, 43)
print("\n")
print(X_train, X_test, y_train, y_test)

X and y:

       Years  Length (KM)  Width (KM)  Country  Latitude  Longtitude  Deaths
0       2022        3.235       0.091        0   40.3790    -75.2266       0
1       2022        0.209       0.046        1   38.9196    -77.2259       0
2       2022        0.080       0.027        1   38.8653    -77.4579       0
3       2022       14.709       0.457        0   41.1600    -76.7300       1
4       2022        0.354       0.091        2   35.8672    -78.8372       0
...      ...          ...         ...      ...       ...         ...     ...
14785   2010        1.271       0.137       10   31.5095    -93.9232       0
14786   2010        0.257       0.023        6   30.8150    -92.0455       0
14787   2010        7.435       0.023       29   33.7160   -118.1250       0
14788   2010        0.225       0.009       29   34.4248   -119.8770       0
14789   2010        0.853       0.014       29   36.7300   -119.8600       0

[14790 rows x 7 columns] 0        1
1        0
2        0
3      

**Machine learning with use in LinearRegression**

In [8]:
# This function return the trained_model after use in LinearRegression algorithm
def train1stModelLinearRegression(X_train, y_train):
    predictor = LinearRegression(n_jobs=-1)
    trained_model = predictor.fit(X_train, y_train)
    return trained_model

In [9]:
# This function print the accuracy 
def LinearRegressionAlgo(target_col_name,X_train, X_test, y_train, y_test):
    trained_model = train1stModelLinearRegression(X_train, y_train)
    pred_value = predictSt(trained_model, X_test)
    y_pred = pd.Series(pred_value,index=X_test.index)
    evaluate_value = r2_score(y_test, y_pred)
    print(evaluate_value)

**The result after LinearRegression Algorithm:**

In [10]:
LinearRegressionAlgo(target_col_name,X_train, X_test, y_train, y_test)

0.4099282454405827


We can see that LinearRegression Algorithm gives us worst result, so I try to use in different algorithms.

**Machine learning with use in DecisionTree  Algorithm**

In [11]:
def train2stModelDecisionTreeClassifier(X_train, y_train):
    predictor = DecisionTreeClassifier(max_depth =4)
    trained_model = predictor.fit(X_train, y_train)
    return trained_model

In [12]:
def DecisionTreeClassifierAlgo(target_col_name,X_train, X_test, y_train, y_test):
    trained_model = train2stModelDecisionTreeClassifier(X_train, y_train)
    pred_value = predictSt(trained_model, X_test)
    y_pred = pd.Series(pred_value,index=X_test.index)
    evaluate_value = accuracy_score(y_test, y_pred)
    print(evaluate_value)

**The result after DecisionTree Algorithm:**

In [13]:
DecisionTreeClassifierAlgo(target_col_name,X_train, X_test, y_train, y_test)

0.664412891593419


**Machine learning with use in KNeighborsClassifier Algorithm**

In [14]:
def train3stModelKNeighborsClassifier(X_train, y_train):
    predictor = KNeighborsClassifier(n_neighbors = 15)
    trained_model = predictor.fit(X_train, y_train)
    return trained_model

In [15]:
def KNeighborsClassifierAlgo(target_col_name,X_train, X_test, y_train, y_test):
    trained_model = train3stModelKNeighborsClassifier(X_train, y_train)
    pred_value = predictSt(trained_model, X_test)
    y_pred = pd.Series(pred_value,index=X_test.index)
    evaluate_value = accuracy_score(y_test, y_pred)
    print(evaluate_value)

**The result after KNN Algorithm:**

In [16]:
KNeighborsClassifierAlgo(target_col_name,X_train, X_test, y_train, y_test)

0.6294793779580797


**Machine learning with use in RandomForest Algorithm**

In [17]:
def train4stModelRandomForestClassifier(X_train, y_train):
    predictor = RandomForestClassifier(n_estimators=51)
    trained_model = predictor.fit(X_train, y_train)
    return trained_model

In [18]:
def RandomForestClassifierAlgo(target_col_name,X_train, X_test, y_train, y_test):
    trained_model = train4stModelRandomForestClassifier(X_train, y_train)
    pred_value = predictSt(trained_model, X_test)
    y_pred = pd.Series(pred_value,index=X_test.index)
    evaluate_value = accuracy_score(y_test, y_pred)
    print(evaluate_value)

**The result after RandomForest Algorithm:**

In [19]:
RandomForestClassifierAlgo(target_col_name,X_train, X_test, y_train, y_test)

0.6864998873112463


So, We can see that RandomForest Algorithm gives us the best accuracy with 69% success